#### Create BQ dataset for storing the intermediate data

In [ ]:
model_name = "gemini-2.0-flash-001"
dataset = "product_data_int"
region = "us-central1"
project_id = "amazon-product-reviews-452322"
region = "us-central1"

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=project_id, location=region)

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset amazon-product-reviews-452322.product_data_int


# Loading all tables into INT area

## prime_users Table

In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.Prime_Users` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.prime_users`;


Query is running:   0%|          |

""


## product_image_data_stg Table

In [ ]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.Product_Image_Data` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_image_data_stg`;


Query is running:   0%|          |

""


## product_meta_data Table

In [ ]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.Product_Meta_Data` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_data`;


Query is running:   0%|          |

""


## product_meta_data_2 Table

In [ ]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.Product_Meta_Data_2` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_meta_data_2`;


Query is running:   0%|          |

""


## product_reviews Table

In [ ]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.Product_Reviews` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.product_reviews`;


Query is running:   0%|          |

""


##  user Table

In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.User` AS
SELECT
    SAFE_CAST(user_id[OFFSET(0)] AS INT64) AS user_id,
    user_name[OFFSET(0)] AS username,
    SAFE_CAST(review_id[OFFSET(0)] AS INT64) AS review_id,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_stg.user`;


Query is running:   0%|          |

""


## user_reviews Table

In [ ]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.User_Reviews` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.user_reviews`;


Query is running:   0%|          |

""


## categories Table

In [ ]:
%%bigquery --project amazon-product-reviews-452322
CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.Categories` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_stg.categories`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322
SELECT table_name
FROM `product_data_int`.INFORMATION_SCHEMA.TABLES;


Query is running:   0%|          |

Downloading:   0%|          |

,table_name
0,Membership_Info
1,tmp_product_meta_data_2
2,Product_Image_Data
3,tmp_membership_info
4,Product_Meta_Data_2
5,User
6,User_Reviews
7,Prime_Users
8,Categories
9,tmp_user_info


# Critieria 8 & 9
8) It can be seen that there are various columns which contains lists and objects embedded as strings. We will convert these into native array types and save as a separate column. Note: We dont do this for some other columns, like images, videos, categories, as they are already in mode = REPEATED in BigQuery, meaning they are recognized as arrays already in BQ.

9) two different identifier systems to refer to the same entity
There are multiple examples of this shown in the reviews data that we have. We used two different datasets to get the reviews data one from hugging face and one from sales data from kaggle. These two difference sources use two different columns to represent the same reviews id and other important similar cols.

## Fixing prime user first

We will change prime user to the logical entities and make sure they also point to the other User entity.

In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.tmp_user` AS
SELECT
    user_id,
    username,
    NULL AS review_id,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.Prime_Users`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.tmp_user_info` AS
SELECT
    user_id,
    name,
    email_address,
    date_of_birth,
    gender,
    location,
    usage_frequency,
    purchase_history,
    favorite_genres,
    devices_used,
    engagement_metrics,
    feedback_ratings,
    customer_support_interactions,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.Prime_Users`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.tmp_membership_info` AS
SELECT
    user_id,
    membership_start_date,
    membership_end_date,
    subscription_plan,
    payment_information,
    renewal_status,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.Prime_Users`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.User_Info` AS
SELECT
    user_id,
    name,
    email_address,
    date_of_birth,
    gender,
    location,
    usage_frequency,
    purchase_history,
    favorite_genres,
    devices_used,
    engagement_metrics,
    feedback_ratings,
    customer_support_interactions,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.tmp_user_info`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.Membership_Info` AS
SELECT
    user_id,
    membership_start_date,
    membership_end_date,
    subscription_plan,
    payment_information,
    renewal_status,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.tmp_membership_info`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322

INSERT INTO `amazon-product-reviews-452322.product_data_int.User` (
    user_id,
    username,
    review_id,
    _data_source,
    _load_time
)
SELECT
    user_id,
    username,
    review_id,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.tmp_user`;


Query is running:   0%|          |

""


## 2nd Fix: Product Meta Data 2

In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2` AS
SELECT
    product_id,
    product_name,
    brand_name,
    asin,
    category,
    upc_ean_code,
    list_price,
    selling_price,
    quantity,
    model_number,
    about_product,
    product_specification,
    technical_details,
    shipping_weight,
    product_dimensions,
    image_urls,
    product_url,
    variants,
    sku,
    stock,
    product_details,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_stg.product_meta_data_2`

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322

SELECT
   COUNT(product_id) AS product_id_count,
   COUNT(product_name) AS product_name_count,
   COUNT(brand_name) AS brand_name_count,
   COUNT(asin) AS asin_count,
   COUNT(category) AS category_count,
   COUNT(upc_ean_code) AS upc_ean_code_count,
   COUNT(list_price) AS list_price_count,
   COUNT(selling_price) AS selling_price_count,
   COUNT(quantity) AS quantity_count,
   COUNT(model_number) AS model_number_count,
   COUNT(about_product) AS about_product_count,
   COUNT(product_specification) AS product_specification_count,
   COUNT(technical_details) AS technical_details_count,
   COUNT(shipping_weight) AS shipping_weight_count,
   COUNT(product_dimensions) AS product_dimensions_count,
   COUNT(image_urls) AS image_urls_count,
   COUNT(product_url) AS product_url_count,
   COUNT(variants) AS variants_count,
   COUNT(sku) AS sku_count,
   COUNT(stock) AS stock_count,
   COUNT(product_details) AS product_details_count,
   COUNT(*) AS total_rows
FROM `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`

Query is running:   0%|          |

Downloading:   0%|          |

,product_id_count,product_name_count,brand_name_count,asin_count,category_count,upc_ean_code_count,list_price_count,selling_price_count,quantity_count,model_number_count,...,technical_details_count,shipping_weight_count,product_dimensions_count,image_urls_count,product_url_count,variants_count,sku_count,stock_count,product_details_count,total_rows
0,10002,10002,0,0,10002,3919,0,10002,0,10002,...,10002,10002,8921,10002,10002,9992,0,0,0,10002


In [ ]:
%%bigquery --project amazon-product-reviews-452322

SELECT
   product_id,
   product_name,
   list_price,
   selling_price,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`

LIMIT 1000

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_name,list_price,selling_price,_data_source,_load_time
0,ea5c9c223405080d4e865874e5397333,Pacific Play Tents Tri-Color Dome Tent and Tun...,None,$67.69,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
1,0f4430a71110c64ce27259862b79b778,Rubie's Costume Men's Batman Arkham City Adult...,None,$18.00,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
2,8481dbe38041664c8ba70d3a3e9852d7,Gila PFW486 Privacy Frosted Film-48 x6.5' Wind...,None,$9.99,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
3,0b9df9bc7f554b556776ce0cf62debfa,Turtle Beach XO Three Gaming Headset for Xbox One,None,$ 53 98 $59.95 #listPriceLegalMessageText { ma...,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
4,51de7893169e1d84a7c81bbb6f9c2289,Walthers Cornerstone Series Kit HO Scale McGra...,None,$44.74,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
...,...,...,...,...,...,...
995,3fdf9833417ca637903e238f67691558,"Crayola 04-2532 Inspiration Art Case, Multicolor",None,$41.36,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
996,3ce5d15097bf835b20b9f49d9b126d24,"Steiff Sleep Well Bear Grip Toy Plush, Pink",None,$17.93,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
997,5fc8a1626a1fb8314397903460114365,Disney Princess Royal Shimmer Ariel,None,$8.88,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
998,ac8cf721effe9323849f301c14e236dd,"Inky Antics Mounted Rubber Stamp 2.5""X3.75""-Fa...",None,$9.60,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00


In [ ]:
%%bigquery --project amazon-product-reviews-452322

-- First, update the table to move list_price to product_specification
UPDATE `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`
SET product_specification = CONCAT(IFNULL(product_specification, ''),
                                  IFNULL(CONCAT('List Price: ', list_price), ''))
WHERE list_price IS NOT NULL;

-- Then, clean the selling_price column
UPDATE `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`
SET selling_price = REGEXP_EXTRACT(selling_price, r'(\d+\.\d+)')
WHERE selling_price IS NOT NULL AND REGEXP_CONTAINS(selling_price, r'\d+\.\d+');

-- For entries like "$ 14 94", convert to proper decimal format
UPDATE `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`
SET selling_price = REGEXP_REPLACE(REGEXP_REPLACE(selling_price, r'[^\d ]', ''), r'(\d+)\s+(\d+)', r'\1.\2')
WHERE selling_price IS NOT NULL AND REGEXP_CONTAINS(selling_price, r'\d+\s+\d+');

-- Handle price ranges by taking the first price in the range
UPDATE `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`
SET selling_price = REGEXP_EXTRACT(selling_price, r'(\d+\.\d+)')
WHERE selling_price IS NOT NULL AND REGEXP_CONTAINS(selling_price, r'-');

SELECT
   product_id,
   product_name,
   list_price,
   selling_price,
   product_specification
FROM `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`
LIMIT 1000

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_name,list_price,selling_price,product_specification
0,d5c9189f24aadd9d926ef767c9367069,"Williams by Bachmann E-Z Street 2.5"" Curve To ...",None,20.75,
1,53d854e4b2a4a029c11330779e8de033,RoomMates Despicable Me 2 Peel And Stick Wall ...,None,5.99,
2,55891700b1f40f04c817720c6440b6e4,"Munchkin Ocean Squirts Bath Toy, 8 pack",None,8.99,
3,0811c593514ef345df62b4bbd241d803,Connelly Party Cove Island Water Mat,None,249.99,Shipping Information: View shipping rates and ...
4,62c46073c0ffe65f548afe00d07ff6dc,MightySkins Skin Compatible with DJI Mavic Pro...,None,8.99,
...,...,...,...,...,...
995,d0e95ec75de54b5cc32ae6fa37af43ee,amscan Spider-Man Webbed Wonder Folded Loot Ba...,None,4.76,ProductDimensions:8x6.9x0.1inches|ItemWeight:1...
996,7533b642e403b61cecad3e1da4219f14,"Vallejo Game Color Steel Grey Paint, 17ml",None,6.95,ProductDimensions:0.8x0.8x3inches|ItemWeight:1...
997,e4eef1dbb2aeb713f9d6b181b18f5871,30th Birthday Flashing Tiara,None,12.01,ProductDimensions:6x6.2x4.5inches|ItemWeight:0...
998,13b26e6348f33060f92bfa9b309e90da,3B Scientific V2041U Bacteria Anatomical Chart...,None,21.00,Item Weight: 7 ounces |Shipping Wei...


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2` AS
SELECT
    product_id,
    product_name,
    brand_name,
    asin,
    category,
    SAFE_CAST(list_price AS FLOAT64) AS list_price,
    SAFE_CAST(selling_price AS FLOAT64) AS selling_price,
    quantity,
    model_number,
    about_product,
    product_specification,
    technical_details,
    shipping_weight,
    product_dimensions,
    image_urls,
    product_url,
    sku,
    product_details,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`
WHERE selling_price IS NOT NULL
AND SAFE_CAST(selling_price AS FLOAT64) IS NOT NULL;

-- View the results
SELECT
    product_id,
    product_name,
    list_price,
    selling_price,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`
LIMIT 1000;

Query is running:   0%|          |

Downloading:   0%|          |

,product_id,product_name,list_price,selling_price,_data_source,_load_time
0,f89c25d355a61d4977af027e0f7ed7d3,Premier Energizer AUX Cable Auxiliary Audio Co...,NaN,7.49,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
1,b6d1999b1d2f9c39f577ec402be27e95,PDP Pixel Pals Kingdom Hearts Goofy Collectibl...,NaN,12.63,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
2,4cb99db21448501f365ee96ebc4630ea,Shimmer and Shine Baby Girls' Toddler Costume ...,NaN,23.00,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
3,3da06c49c9889d70cc1ecf47a5055aff,Ferrari Classic Roller Skates,NaN,139.99,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
4,43181023ff436ab218b5beb20d7117cc,Mattel DC Comics Multiverse Suicide Squad Figu...,NaN,26.74,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
...,...,...,...,...,...,...
995,0860fc6e739295a7c8ff37855a3b5cf3,Stephan Baby Plush Knotty Animal Security Blan...,NaN,8.99,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
996,9dd02dd0aee57c56d67defa7dbccad35,Manhattan Toy Zoo Winkel Lion Rattle and Senso...,NaN,12.26,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
997,2d14fe61a02de42e703908ca347ec875,"MOLUK Nigi, Nagi, Nogi - Primary Colors",NaN,13.79,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00
998,eebdddc7884b118b808cc4a0fbe66277,"Skip Hop Stay Cool Teether, BPA-free and Freez...",NaN,7.99,initial_loads/product_meta_data_2,2025-03-01 23:28:09.694386+00:00


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.Product_Meta_Data_2` AS
SELECT
    product_id,
    product_name,
    brand_name,
    asin,
    category,
    list_price,
    selling_price,
    quantity,
    model_number,
    about_product,
    product_specification,
    technical_details,
    shipping_weight,
    product_dimensions,
    image_urls,
    product_url,
    sku,
    product_details,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322


-- Step 2: Retrieve all column names and data types from Product_Meta_Data_2
SELECT column_name, data_type
FROM amazon-product-reviews-452322.product_data_int.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'Product_Meta_Data_2';


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,product_id,STRING
1,product_name,STRING
2,brand_name,STRING
3,asin,STRING
4,category,STRING
5,list_price,FLOAT64
6,selling_price,FLOAT64
7,quantity,INT64
8,model_number,STRING
9,about_product,STRING


In [ ]:
%%bigquery --project amazon-product-reviews-452322

SELECT column_name, data_type
FROM amazon-product-reviews-452322.product_data_int.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'Product_Meta_Data';


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,main_category,STRING
1,title,STRING
2,average_rating,FLOAT64
3,rating_number,INT64
4,features,STRING
5,description,STRING
6,price,FLOAT64
7,store,STRING
8,images,ARRAY<STRING>
9,videos,ARRAY<STRING>


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE amazon-product-reviews-452322.product_data_int.tmp_pmd2 AS
SELECT
    category AS main_category,
    product_name AS title,
    NULL AS average_rating,
    NULL AS rating_number,
    product_specification AS features,
    about_product AS description,
    selling_price AS price,
    NULL AS store,
    image_urls AS images,
    NULL AS videos,
    NULL AS categories,
    CONCAT(
        IFNULL(product_details, ''),
        ' | brand_name: ', IFNULL(brand_name, ''),
        ' | list_price: ', IFNULL(CAST(list_price AS STRING), ''),
        ' | quantity: ', IFNULL(CAST(quantity AS STRING), ''),
        ' | model_number: ', IFNULL(model_number, ''),
        ' | technical_details: ', IFNULL(technical_details, ''),
        ' | shipping_weight: ', IFNULL(shipping_weight, ''),
        ' | product_dimensions: ', IFNULL(product_dimensions, ''),
        ' | product_url: ', IFNULL(product_url, ''),
        ' | sku: ', IFNULL(sku, '')
    ) AS details,
    asin AS parent_asin,
    NULL AS bought_together,
    _data_source,
    _load_time
FROM amazon-product-reviews-452322.product_data_int.Product_Meta_Data_2;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322


-- Step 2: Retrieve all column names and data types from Product_Meta_Data_2
SELECT column_name, data_type
FROM amazon-product-reviews-452322.product_data_int.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'tmp_pmd2';


Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,main_category,STRING
1,title,STRING
2,average_rating,INT64
3,rating_number,INT64
4,features,STRING
5,description,STRING
6,price,FLOAT64
7,store,INT64
8,images,ARRAY<STRING>
9,videos,INT64


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE amazon-product-reviews-452322.product_data_int.Product_Meta_Data_Merged AS
SELECT * FROM amazon-product-reviews-452322.product_data_int.Product_Meta_Data
UNION ALL
SELECT
    CAST(main_category AS STRING),
    CAST(title AS STRING),
    CAST(average_rating AS FLOAT64),
    CAST(rating_number AS INT64),
    CAST(features AS STRING),
    CAST(description AS STRING),
    CAST(price AS FLOAT64),
    CAST(store AS STRING),
    images,
    ARRAY<STRING>[IFNULL(CAST(videos AS STRING), '')] AS videos,
    ARRAY<STRING>[IFNULL(CAST(categories AS STRING), '')] AS categories,
    CAST(details AS STRING),
    CAST(parent_asin AS STRING),
    CAST(bought_together AS STRING),
    CAST(_data_source AS STRING),
    CAST(_load_time AS TIMESTAMP)
FROM amazon-product-reviews-452322.product_data_int.tmp_pmd2;


Query is running:   0%|          |

""


# Critieria 10

Here we are going to split the users table up to the proper entities. The users table contains entities such as prime_users, membership_details, user_history,


Prime Users (prime_users): Users with a Prime membership.

Membership Details (membership_details): Information about membership duration, status, etc.

User History (user_history): Purchase history, review history, etc.

In [ ]:
%%bigquery --project amazon-product-reviews-452322
SELECT * FROM `product_data_int.User` LIMIT 10;


Query is running:   0%|          |

Downloading:   0%|          |

,user_id,username,review_id,_data_source,_load_time
0,72,aanderson,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
1,1196,aaron27,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
2,1442,aaron29,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
3,2428,aaron39,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
4,438,aaron70,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
5,2170,aaronking,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
6,1094,aaronmiller,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
7,1331,aaronwelch,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
8,1829,aatkinson,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00
9,1290,abigailconrad,<NA>,initial_loads/prime_users,2025-03-01 23:25:54.058916+00:00


In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data` AS
SELECT
    main_category,
    product_name AS title,
    SAFE_DIVIDE(rating, rating_count) AS average_rating,
    CAST(rating_count AS INT64) AS rating_number,
    CAST(NULL AS STRING) AS features,                   
    about_product AS description,                          
    CAST(actual_price AS FLOAT64) AS price,
    CAST(NULL AS STRING) AS store,                          
    ARRAY[img_link] AS images,                              
    CAST(NULL AS ARRAY<STRING>) AS videos,                  
    ARRAY[ARRAY_TO_STRING(ARRAY[
        main_category,
        subcategory_1,
        subcategory_2,
        subcategory_3,
        subcategory_4,
        subcategory_5,
        subcategory_6,
        subcategory_7,
        subcategory_8
    ], ' > ')] AS categories,                              
    CONCAT(
        'Discounted Price: ', CAST(discounted_price AS STRING),
        ', Actual Price: ', CAST(actual_price AS STRING),
        ', Discount Percentage: ', CAST(discount_percentage AS STRING),
        ', Product Link: ', product_link
    ) AS details,                                           
    CAST(product_id AS STRING) AS parent_asin,              
    CAST(NULL AS STRING) AS bought_together,                
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.User`;


Executing query with job ID: 019b4fd8-4604-452c-b799-7a170e04014a
Query executing: 0.26s


ERROR:
 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/amazon-product-reviews-452322/queries/019b4fd8-4604-452c-b799-7a170e04014a?maxResults=0&location=us-central1&prettyPrint=false: Unrecognized name: main_category at [3:5]

Location: us-central1
Job ID: 019b4fd8-4604-452c-b799-7a170e04014a



In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.tmp_product_reviews` AS
SELECT
    rating,
    ARRAY_TO_STRING(review_title, ', ') AS title,     
    ARRAY_TO_STRING(review_content, ', ') AS text,    
    ARRAY[img_link] AS images,                        
    CAST(product_id AS STRING) AS asin,            
    CAST(product_id AS STRING) AS parent_asin,       
    ARRAY_TO_STRING(user_id, ', ') AS user_id,       
    CAST(NULL AS DATE) AS review_date,               
    CAST(NULL AS INT64) AS helpful_vote,              
    CAST(NULL AS BOOL) AS verified_purchase,         
    CAST(NULL AS STRING) AS details,               
    CAST(NULL AS ARRAY<STRING>) AS videos,           
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.User`;


Executing query with job ID: 282795dd-d29b-4e2e-b82d-ae8a2f0f896f
Query executing: 0.29s


ERROR:
 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/amazon-product-reviews-452322/queries/282795dd-d29b-4e2e-b82d-ae8a2f0f896f?maxResults=0&location=us-central1&prettyPrint=false: Unrecognized name: rating at [3:5]

Location: us-central1
Job ID: 282795dd-d29b-4e2e-b82d-ae8a2f0f896f



In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.tmp_users` AS
SELECT
    user_id,
    user_name as username,
    review_id,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.User`;


Executing query with job ID: ee6c7c30-3cc1-4e92-8494-c75dda1246f2
Query executing: 0.25s


ERROR:
 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/amazon-product-reviews-452322/queries/ee6c7c30-3cc1-4e92-8494-c75dda1246f2?maxResults=0&location=us-central1&prettyPrint=false: Unrecognized name: user_name; Did you mean username? at [4:5]

Location: us-central1
Job ID: ee6c7c30-3cc1-4e92-8494-c75dda1246f2



## Change the tmp table to permanent tables

In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.User` AS
SELECT * FROM `amazon-product-reviews-452322.product_data_int.tmp_users`;

Executing query with job ID: a2928146-867a-4614-b1d8-78eca65b216b
Query executing: 0.17s


ERROR:
 404 Not found: Table amazon-product-reviews-452322:product_data_int.tmp_users was not found in location us-central1; reason: notFound, message: Not found: Table amazon-product-reviews-452322:product_data_int.tmp_users was not found in location us-central1

Location: us-central1
Job ID: a2928146-867a-4614-b1d8-78eca65b216b



In [ ]:
%%bigquery --project amazon-product-reviews-452322

INSERT INTO `amazon-product-reviews-452322.product_data_int.Product_Reviews`
SELECT * FROM `amazon-product-reviews-452322.product_data_int.tmp_product_reviews`;


Executing query with job ID: 84b5ad7a-4b7e-451b-859a-aacf67cd860a
Query executing: 0.21s


ERROR:
 404 Not found: Table amazon-product-reviews-452322:product_data_int.tmp_product_reviews was not found in location us-central1; reason: notFound, message: Not found: Table amazon-product-reviews-452322:product_data_int.tmp_product_reviews was not found in location us-central1

Location: us-central1
Job ID: 84b5ad7a-4b7e-451b-859a-aacf67cd860a



In [ ]:
%%bigquery --project amazon-product-reviews-452322

INSERT INTO `amazon-product-reviews-452322.product_data_int.Product_Meta_Data`
SELECT * FROM `amazon-product-reviews-452322.product_data_int.tmp_product_meta_data`;


Executing query with job ID: 80bf7e90-d48b-470b-952e-6ac56c526f19
Query executing: 0.23s


ERROR:
 404 Not found: Table amazon-product-reviews-452322:product_data_int.tmp_product_meta_data was not found in location us-central1; reason: notFound, message: Not found: Table amazon-product-reviews-452322:product_data_int.tmp_product_meta_data was not found in location us-central1

Location: us-central1
Job ID: 80bf7e90-d48b-470b-952e-6ac56c526f19



# Dropping Tables to clean up



*  Prime User
*  PMD
*  PMD2  
*  tmp_membership_info
*  tmp_product_meta_data_2
*  tmp_user
*  tmp_user_info




In [ ]:
%%bigquery --project amazon-product-reviews-452322

DROP TABLE IF EXISTS amazon-product-reviews-452322.product_data_int.Prime_Users;
DROP TABLE IF EXISTS amazon-product-reviews-452322.product_data_int.PMD;
DROP TABLE IF EXISTS amazon-product-reviews-452322.product_data_int.tmp_pmd2;
DROP TABLE IF EXISTS amazon-product-reviews-452322.product_data_int.tmp_membership_info;
DROP TABLE IF EXISTS amazon-product-reviews-452322.product_data_int.tmp_product_meta_data_2;
DROP TABLE IF EXISTS amazon-product-reviews-452322.product_data_int.tmp_user;
DROP TABLE IF EXISTS amazon-product-reviews-452322.product_data_int.tmp_user_info;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322

SELECT column_name, data_type
FROM amazon-product-reviews-452322.product_data_int.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'Product_Meta_Data_Merged';

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,main_category,STRING
1,title,STRING
2,average_rating,FLOAT64
3,rating_number,INT64
4,features,STRING
5,description,STRING
6,price,FLOAT64
7,store,STRING
8,images,ARRAY<STRING>
9,videos,ARRAY<STRING>


In [ ]:
%%bigquery --project amazon-product-reviews-452322

SELECT column_name, data_type
FROM amazon-product-reviews-452322.product_data_int.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'Product_Reviews';

Query is running:   0%|          |

Downloading:   0%|          |

,column_name,data_type
0,rating,FLOAT64
1,title,STRING
2,text,STRING
3,images,ARRAY<STRING>
4,asin,STRING
5,parent_asin,STRING
6,user_id,STRING
7,review_date,DATE
8,helpful_vote,INT64
9,verified_purchase,BOOL


# Fix

In [ ]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE `amazon-product-reviews-452322.product_data_int.User` AS
SELECT
    CAST(user_id AS STRING) AS user_id,
    username,
    review_id,
    _data_source,
    _load_time
FROM `amazon-product-reviews-452322.product_data_int.User`;


Query is running:   0%|          |

""


In [ ]:
%%bigquery --project amazon-product-reviews-452322

DROP TABLE IF EXISTS `amazon-product-reviews-452322.product_data_int.User_Reviews`;

CREATE TABLE `amazon-product-reviews-452322.product_data_int.User_Reviews` (
    user_id STRING NOT NULL,
    review_id STRING NOT NULL,
    _data_source STRING,
    _load_time TIMESTAMP
);


Query is running:   0%|          |

""
